In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk
import re

import string
import collections

#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import preprocessing
import lightgbm
import pickle 

In [2]:
#Lightgbm with CV. 
data=pd.read_csv('dataset_00_with_header.csv')

#Remove constant-valued features (x067, x094, x095, x096): no prediction power; 
constant_feat=[]
for i in data.columns:
    a=data[i].nunique()
    if a==1:
        constant_feat.append(i)
print(constant_feat)
        
#Remove 4 features w/ only 1 constant value; 
data.drop(constant_feat, axis=1, inplace=True)

['x067', 'x094', 'x095', 'x096']


In [5]:
#Cross validation; 
from sklearn.model_selection import KFold
X=data.drop('y', axis=1)
Y=data['y']
kf = KFold(n_splits=5, random_state=100, shuffle=False)
kf.get_n_splits(X)

ytest_pred=[]
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    xtrain, xtest = X.iloc[train_index], X.iloc[test_index]
    ytrain, ytest = Y.iloc[train_index], Y.iloc[test_index]

    #Imputation;   
    xtrain.fillna(xtrain.median(), inplace=True)
    xtest.fillna(xtrain.median(), inplace=True)
    
    '''
    #RandomForestRegressor; 
    regr = RandomForestRegressor(random_state=222, n_estimators=100)
    regr.fit(xtrain, ytrain)
    '''
    
    #Lightgbm Regressor; 
    params = {'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': 'rmse',
          'max_depth': 10, 
          'learning_rate': 0.1,
          'early_stopping_round': 100}
    n_estimators = 10000    
    d_train = lightgbm.Dataset(xtrain, label=ytrain)
    d_test = lightgbm.Dataset(xtest, label=ytest)
    watchlist = [d_train, d_test]
    model = lightgbm.train(params=params, train_set=d_train, num_boost_round=n_estimators, valid_sets=watchlist, verbose_eval=10)
    
    ytest_pred.append(model.predict(xtest))

TRAIN: [20000 20001 20002 ... 99997 99998 99999] TEST: [    0     1     2 ... 19997 19998 19999]


C:\Users\Yunsun2\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\Yunsun2\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[10]	training's rmse: 57.0727	valid_1's rmse: 57.5204
[20]	training's rmse: 38.1593	valid_1's rmse: 38.7402
[30]	training's rmse: 32.2472	valid_1's rmse: 32.9235
[40]	training's rmse: 29.9461	valid_1's rmse: 30.7471
[50]	training's rmse: 28.7361	valid_1's rmse: 29.6531
[60]	training's rmse: 27.9636	valid_1's rmse: 28.9811
[70]	training's rmse: 27.3798	valid_1's rmse: 28.4995
[80]	training's rmse: 26.9489	valid_1's rmse: 28.1688
[90]	training's rmse: 26.5384	valid_1's rmse: 27.858
[100]	training's rmse: 26.2065	valid_1's rmse: 27.62
[110]	training's rmse: 25.9172	valid_1's rmse: 27.4407
[120]	training's rmse: 25.6746	valid_1's rmse: 27.3106
[130]	training's rmse: 25.4445	valid_1's rmse: 27.1939
[140]	training's rmse: 25.2166	valid_1's rmse: 27.0626
[150]	training's rmse: 25.0197	valid_1's rmse: 26.9587
[160]	training's rmse: 24.8586	valid_1's rmse: 26.9177
[170]	training's rmse: 24.6864	valid_1's rmse: 26.8412
[180]	training

[1490]	training's rmse: 15.4091	valid_1's rmse: 25.2491
[1500]	training's rmse: 15.3688	valid_1's rmse: 25.2472
[1510]	training's rmse: 15.3232	valid_1's rmse: 25.2416
[1520]	training's rmse: 15.2869	valid_1's rmse: 25.2381
[1530]	training's rmse: 15.2546	valid_1's rmse: 25.2367
[1540]	training's rmse: 15.2038	valid_1's rmse: 25.233
[1550]	training's rmse: 15.1561	valid_1's rmse: 25.2248
[1560]	training's rmse: 15.1119	valid_1's rmse: 25.2262
[1570]	training's rmse: 15.0727	valid_1's rmse: 25.227
[1580]	training's rmse: 15.0286	valid_1's rmse: 25.226
[1590]	training's rmse: 14.9943	valid_1's rmse: 25.2228
[1600]	training's rmse: 14.9509	valid_1's rmse: 25.2206
[1610]	training's rmse: 14.9079	valid_1's rmse: 25.216
[1620]	training's rmse: 14.8731	valid_1's rmse: 25.2164
[1630]	training's rmse: 14.8274	valid_1's rmse: 25.2101
[1640]	training's rmse: 14.7905	valid_1's rmse: 25.2055
[1650]	training's rmse: 14.7528	valid_1's rmse: 25.1999
[1660]	training's rmse: 14.7248	valid_1's rmse: 25.2

C:\Users\Yunsun2\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

[1490]	training's rmse: 15.4916	valid_1's rmse: 25.2615
[1500]	training's rmse: 15.4502	valid_1's rmse: 25.26
[1510]	training's rmse: 15.402	valid_1's rmse: 25.2563
[1520]	training's rmse: 15.3576	valid_1's rmse: 25.2573
[1530]	training's rmse: 15.3279	valid_1's rmse: 25.2581
[1540]	training's rmse: 15.2738	valid_1's rmse: 25.2476
[1550]	training's rmse: 15.2361	valid_1's rmse: 25.2461
[1560]	training's rmse: 15.1983	valid_1's rmse: 25.2465
[1570]	training's rmse: 15.1599	valid_1's rmse: 25.2414
[1580]	training's rmse: 15.1198	valid_1's rmse: 25.2451
[1590]	training's rmse: 15.0915	valid_1's rmse: 25.2423
[1600]	training's rmse: 15.06	valid_1's rmse: 25.2377
[1610]	training's rmse: 15.0157	valid_1's rmse: 25.2361
[1620]	training's rmse: 14.973	valid_1's rmse: 25.2293
[1630]	training's rmse: 14.9359	valid_1's rmse: 25.2269
[1640]	training's rmse: 14.9022	valid_1's rmse: 25.2295
[1650]	training's rmse: 14.8657	valid_1's rmse: 25.226
[1660]	training's rmse: 14.826	valid_1's rmse: 25.2197


C:\Users\Yunsun2\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[10]	training's rmse: 57.0896	valid_1's rmse: 57.59
[20]	training's rmse: 38.2338	valid_1's rmse: 38.893
[30]	training's rmse: 32.2446	valid_1's rmse: 33.0541
[40]	training's rmse: 29.9946	valid_1's rmse: 30.9469
[50]	training's rmse: 28.8162	valid_1's rmse: 29.8881
[60]	training's rmse: 28.0463	valid_1's rmse: 29.2244
[70]	training's rmse: 27.4915	valid_1's rmse: 28.7779
[80]	training's rmse: 27.0103	valid_1's rmse: 28.4024
[90]	training's rmse: 26.607	valid_1's rmse: 28.1285
[100]	training's rmse: 26.2656	valid_1's rmse: 27.9008
[110]	training's rmse: 25.9852	valid_1's rmse: 27.7307
[120]	training's rmse: 25.7224	valid_1's rmse: 27.5764
[130]	training's rmse: 25.5045	valid_1's rmse: 27.4685
[140]	training's rmse: 25.2975	valid_1's rmse: 27.3701
[150]	training's rmse: 25.0926	valid_1's rmse: 27.2711
[160]	training's rmse: 24.9047	valid_1's rmse: 27.1828
[170]	training's rmse: 24.731	valid_1's rmse: 27.12
[180]	training's r

[1490]	training's rmse: 15.5211	valid_1's rmse: 25.4549
[1500]	training's rmse: 15.4679	valid_1's rmse: 25.443
[1510]	training's rmse: 15.4296	valid_1's rmse: 25.4395
[1520]	training's rmse: 15.3891	valid_1's rmse: 25.4378
[1530]	training's rmse: 15.3538	valid_1's rmse: 25.4378
[1540]	training's rmse: 15.3162	valid_1's rmse: 25.4379
[1550]	training's rmse: 15.2762	valid_1's rmse: 25.4377
[1560]	training's rmse: 15.2311	valid_1's rmse: 25.4337
[1570]	training's rmse: 15.1942	valid_1's rmse: 25.4342
[1580]	training's rmse: 15.1623	valid_1's rmse: 25.4325
[1590]	training's rmse: 15.1246	valid_1's rmse: 25.4299
[1600]	training's rmse: 15.0936	valid_1's rmse: 25.4253
[1610]	training's rmse: 15.051	valid_1's rmse: 25.4213
[1620]	training's rmse: 15.0068	valid_1's rmse: 25.4212
[1630]	training's rmse: 14.9674	valid_1's rmse: 25.4183
[1640]	training's rmse: 14.9345	valid_1's rmse: 25.4144
[1650]	training's rmse: 14.8884	valid_1's rmse: 25.4072
[1660]	training's rmse: 14.8583	valid_1's rmse: 25

[2960]	training's rmse: 11.1022	valid_1's rmse: 25.2362
[2970]	training's rmse: 11.0787	valid_1's rmse: 25.2336
[2980]	training's rmse: 11.0587	valid_1's rmse: 25.2315
[2990]	training's rmse: 11.0405	valid_1's rmse: 25.2315
[3000]	training's rmse: 11.0183	valid_1's rmse: 25.2303
[3010]	training's rmse: 10.9988	valid_1's rmse: 25.2311
[3020]	training's rmse: 10.9738	valid_1's rmse: 25.231
[3030]	training's rmse: 10.9517	valid_1's rmse: 25.2307
[3040]	training's rmse: 10.9325	valid_1's rmse: 25.23
[3050]	training's rmse: 10.9022	valid_1's rmse: 25.2266
[3060]	training's rmse: 10.8714	valid_1's rmse: 25.2262
[3070]	training's rmse: 10.8473	valid_1's rmse: 25.2274
[3080]	training's rmse: 10.8219	valid_1's rmse: 25.2254
[3090]	training's rmse: 10.8002	valid_1's rmse: 25.2251
[3100]	training's rmse: 10.7838	valid_1's rmse: 25.2233
[3110]	training's rmse: 10.7617	valid_1's rmse: 25.2238
[3120]	training's rmse: 10.7432	valid_1's rmse: 25.2257
[3130]	training's rmse: 10.7253	valid_1's rmse: 25.

In [25]:
ypred_list=[]
for i in ytest_pred:
    for j in i:
        ypred_list.append(j)

In [27]:
#Performance evaluation;
from sklearn.metrics import mean_squared_error, r2_score
y_preddummy=(abs(ypred_list-Y)<=3)

RSME=np.sqrt(mean_squared_error(Y, ypred_list)) #change ytest_pred -> ytest_pred_list; 
R2=r2_score(Y, ypred_list)
Accu=y_preddummy.value_counts()/len(y_preddummy)

data = [RSME, R2, Accu[True]]
performance_metric = pd.DataFrame(data, index=['RSME', 'R-squared', 'Accuracy'], columns=['Value']) #smaller, bigger, bigger; 
pd.DataFrame(performance_metric).to_csv('performance_metric.txt', sep='\t', header=True, index=True)

print(performance_metric)
pd.DataFrame(ypred_list).to_csv('y_pred.txt', sep='\t', header=False, index=True)

               Value
RSME       25.070647
R-squared   0.955210
Accuracy    0.159510
